In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from functools import partial
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,
    Rotate
)
AUTOTUNE = tf.data.experimental.AUTOTUNE


Num GPUs Available:  1


**LUYỆN TẬP THỰC HÀNH TENSORFLOW**

[DAY 1] BASIC MODELING

#### 1. Linear regression

Cho 2 mảng xs (biến độc lập) và ys (biến phụ thuộc) như dưới, sử dụng tensorflow hãy xây dựng một mô hình đơn giản thể hiện mối quan hệ của 2 biến này



In [ ]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], dtype=float)


In [ ]:

linear_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])
linear_model.build([None,1])
linear_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [ ]:
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.05),
    loss='mean_absolute_error')

history = linear_model.fit(
    xs,
    ys,
    epochs=200,
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)
pd.DataFrame(history.history).iloc[::10]

,loss,val_loss
0,0.009040,0.219270
10,0.032696,0.074986
20,0.017456,0.037871
30,0.007399,0.027449
40,0.007352,0.037427
50,0.008352,0.002589
60,0.008772,0.002972
70,0.017396,0.021959
80,0.012793,0.007782
90,0.000069,0.037458


In [ ]:
print(linear_model.predict(xs).round(2))

print('\n y = {}*x + {}'.format(str(round(linear_model.layers[0].weights[0].numpy()[0,0],2)),
                               str(round(linear_model.layers[0].bias.numpy()[0],2)))
     )


[[0.  ]
 [1.  ]
 [1.99]
 [2.99]
 [3.99]
 [4.98]]

 y = 1.0*x + 1.0


#### 2. Fashion MNIST

Xây dựng mô hình phân loại với bộ dữ liệu Fashion MNIST.<br>
Chú ý: đầu vào ảnh 28x28, đầu ra: 10 nhãn phân loại


In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

4431872/4422102 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [46]:
def fc_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

# define cnn model
def cnn_model():
	model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(500, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
	# compile model
	model.compile(optimizer='adam',
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
               metrics=['accuracy'])
	return model

def resnet50_model():
	model = tf.keras.applications.resnet50.ResNet50(
        include_top=False, weights='imagenet', input_tensor=None,
        input_shape=(32,32,3), pooling=None, classes=10
        )
	# compile model
	model.compile(optimizer='adam',
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
               metrics=['accuracy'])
	return model


### fc

In [ ]:
# model = fc_model()
# model.fit(train_images, train_labels, epochs=10)
# test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 3.3117 - accuracy: 0.6959
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6951 - accuracy: 0.7505
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5956 - accuracy: 0.7831
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5413 - accuracy: 0.8089
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5142 - accuracy: 0.8208
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4919 - accuracy: 0.8278
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4887 - accuracy: 0.8313
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4926 - accuracy: 0.8294
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4713 - accuracy: 0.8348
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.466

### resnet50

In [42]:
# def convert_types(image, label):
#     image = np.expand_dims(image, axis=3)
#     image = tf.cast(image, tf.float32)
#     image /= 255
#     image = tf.image.resize_with_pad(
#         image,
#         32, 32, 
#         method=tf.image.ResizeMethod.BILINEAR,
#         antialias=False
#     )

#     image = tf.image.grayscale_to_rgb(image)
#     return image, label

# n_train_images, _ = convert_types(train_images, train_labels)
# n_test_images, _ = convert_types(test_images, test_labels)
# n_train_images.shape, n_test_images.shape

In [43]:
# model_v2 = resnet50_model()
# model_v2.summary()

In [44]:
# tf.config.experimental_run_functions_eagerly(True)

# model_v2.fit(n_train_images, train_labels, epochs=10)
# test_loss, test_acc = model_v2.evaluate(n_test_images,  test_labels, verbose=2)


### cnn

In [47]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

n_train_images, _ = convert_types(train_images, train_labels)
n_test_images, _ = convert_types(test_images, test_labels)
del _
n_train_images.shape, n_test_images.shape

(TensorShape([60000, 28, 28]), TensorShape([10000, 28, 28]))

In [48]:
model_v3 = cnn_model()
model_v3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)             

In [49]:
model_v3.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)

model_v3.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 15s 8ms/step - loss: 2.5749 - accuracy: 0.7938
Epoch 2/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.3952 - accuracy: 0.8551
Epoch 3/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.3517 - accuracy: 0.8716
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.3248 - accuracy: 0.8815
Epoch 5/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.2950 - accuracy: 0.8910
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2751 - accuracy: 0.9000
Epoch 7/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.2487 - accuracy: 0.9078
Epoch 8/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.2270 - accuracy: 0.9150
Epoch 9/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2106 - accuracy: 0.9225
Epoch 10/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.1918

In [50]:
model_v3.fit(train_images, train_labels, epochs=10)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)

model_v3.fit(train_images, train_labels, epochs=5)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)

model_v3.fit(train_images, train_labels, epochs=5)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)

model_v3.fit(train_images, train_labels, epochs=5)
test_loss, test_acc = model_v3.evaluate(test_images,  test_labels, verbose=2)


Epoch 1/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0914 - accuracy: 0.9702
Epoch 2/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0745 - accuracy: 0.9754
Epoch 3/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0862 - accuracy: 0.9735
Epoch 4/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0927 - accuracy: 0.9737
Epoch 5/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0836 - accuracy: 0.9740
Epoch 6/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0701 - accuracy: 0.9787
Epoch 7/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0803 - accuracy: 0.9772
Epoch 8/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0785 - accuracy: 0.9776
Epoch 9/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0665 - accuracy: 0.9814
Epoch 10/10
1875/1875 [==============================] - 14s 7ms/step - l